In [2]:
import json
import sqlite3

In [3]:
con = sqlite3.connect("local.db")
cur = con.cursor()

In [7]:
cur.execute("CREATE TABLE Book (\
    abbrev          CHAR (2)  PRIMARY KEY\
                              UNIQUE\
                              NOT NULL,\
    full_name       CHAR (50) NOT NULL,\
    number_in_order INTEGER   UNIQUE\
                              NOT NULL\
);\
")
cur.execute("CREATE TABLE Chapter (\
    id      INTEGER  PRIMARY KEY AUTOINCREMENT\
                     UNIQUE\
                     NOT NULL,\
    book_of CHAR (2) REFERENCES Book (abbrev) \
                     NOT NULL,\
    number  INTEGER  NOT NULL\
);\
")
cur.execute("CREATE TABLE Verse (\
    id         INTEGER PRIMARY KEY AUTOINCREMENT\
                       NOT NULL\
                       UNIQUE,\
    chapter_of INTEGER REFERENCES Chapter (id) \
                       NOT NULL,\
    number     INTEGER NOT NULL,\
    text       TEXT    NOT NULL\
);\
")
con.commit()

In [4]:
with open("ru.json", encoding="utf-8") as f:
    Bible = json.load(f)
with open("transitions.json", encoding="utf-8") as f:
    abbrevs = json.load(f)

In [9]:
for book_num, book in enumerate(Bible):
    cur.execute(f"INSERT INTO Book (abbrev, full_name, number_in_order) VALUES ('{book}', '{abbrevs[book]}', {book_num + 1})")
    for chapter, ch_text in enumerate(Bible[book]):
        cur.execute(f"INSERT INTO Chapter (book_of, number) VALUES ('{book}', {chapter + 1})")
        chapter_of = cur.lastrowid
        for verse, ve_text in enumerate(ch_text):
            cur.execute(f"INSERT INTO Verse (chapter_of, number, text) VALUES ({chapter_of}, {verse + 1}, '{ve_text}')")
con.commit()

In [7]:
result = cur.execute("SELECT v.number, v.text FROM Verse v \
                    WHERE v.chapter_of = (\
                    SELECT c.id FROM Chapter c WHERE c.book_of = 'бт' \
                    AND c.number = 1)").fetchall()
dict(result)

{1: 'В начале сотворил Бог небо и землю.',
 2: 'Земля же была безвидна и пуста, и тьма над бездною, и Дух Божий носился над водою.',
 3: 'И сказал Бог: да будет свет. И сталсвет.',
 4: 'И увидел Бог свет, что он хорош, и отделил Бог свет от тьмы.',
 5: 'И назвал Бог свет днем, а тьму ночью. И был вечер, и было утро: день один.',
 6: 'И сказал Бог: да будет твердь посреди воды, и да отделяет она воду от воды.',
 7: 'И создал Бог твердь, и отделил воду, которая подтвердью, от воды, которая над твердью. И стало так.',
 8: 'И назвал Бог твердь небом. И был вечер, и было утро: день второй.',
 9: 'И сказал Бог: да соберется вода, которая под небом, в одно место, и да явится суша. И стало так.',
 10: 'И назвал Бог сушу землею, а собрание вод назвал морями. И увидел Бог, что это хорошо.',
 11: 'И сказал Бог: да произрастит земля зелень, траву, сеющую семя дерево плодовитое, приносящее по роду своему плод, в котором семя его на земле. И стало так.',
 12: 'И произвела земля зелень, траву, сеющую

In [ ]:
""" Поиск стиха SELECT v.text FROM Verse v WHERE v.chapter_of = (
SELECT c.id FROM Chapter c WHERE c.book_of = 'бт' AND c.number = 1) AND v.number = 1
LIMIT 1600; """

""" Поиск текста -- SELECT v.text FROM Verse v WHERE v.text LIKE '%не бойся%'; """
